In [ ]:
%pip install langchain
%pip install openai
%pip install google-search-results

In [ ]:
serpapi_key = 'SERPAPI_KEY'
open_api_key = 'OPEN_API_KEY'

In [ ]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain 
from langchain.memory import ConversationBufferMemory
from langchain.utilities import WikipediaAPIWrapper 
from langchain.llms import OpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent, Tool
from langchain.agents import load_tools
from langchain.agents import AgentType

In [ ]:
# define the llm model

llm = OpenAI(openai_api_key=open_api_key, temperature=0.8) 

In [ ]:
# Define your job-to-be-done and customer to be explored

customer_description = 'a family consisting of two adults and two children with average income who are renting anu apartment in a big city'
job_to_be_done = 'monitoring and paying energy bills'

In [ ]:
# Prompt hardest part of job to be done for customer
hardest_template = PromptTemplate(
    input_variables = ['customer_description','job_to_be_done'], 
    template='You are {customer_description}. What is the hardest part about {job_to_be_done}?')

hardest_chain = LLMChain(llm=llm, prompt=hardest_template, verbose=True, output_key='hardest_part')

hardest = hardest_chain.run(customer_description=customer_description, job_to_be_done=job_to_be_done)

print(hardest)

In [ ]:
# Prompt value proposition to solve hardest part of job-to-be-done

value_proposition_template = PromptTemplate(
    input_variables = ['hardest_part'], 
    template='Create a unique value proposition for a startup that solves the problem of {hardest_part}')

value_proposition_chain = LLMChain(llm=llm, prompt=value_proposition_template, verbose=True, output_key='value_proposition')

valueproposition = value_proposition_chain.run(hardest_part=hardest)

print(valueproposition)


In [ ]:
# Prompt description value proposition canvas

canvas_template = PromptTemplate(
    input_variables = ['value_proposition'], 
    template='Provide the value proposition canvas for {value_proposition}. You should describe the following elements: customer jobs, custome pains, customer gains, products and services, pain relievers, gain creators')

canvas_chain = LLMChain(llm=llm, prompt=canvas_template, verbose=True, output_key='canvas')

canvas = canvas_chain.run(value_proposition=valueproposition)

print(canvas)



In [ ]:
# Prompt identification of core competitors for this value proposition

competitors_template = PromptTemplate(
    input_variables = ['value_proposition'], 
    template='Find three existing startup companies that have a description that is very similar to the following description: {value_proposition}. For each startup, provide company name, location and website.')

competitors_chain = LLMChain(llm=llm, prompt=competitors_template, verbose=True, output_key='competitors')

competitors = competitors_chain.run(value_proposition=valueproposition)

print(competitors)

In [ ]:
# Prompt collecting additional information on competitors via SerpAPI


competitorsselect = competitors[5:]

# split string based on .

competitors_list = competitorsselect.split('\n')

competitors_list

# get informatin for competitors

search = SerpAPIWrapper(serpapi_api_key = serpapi_key)
search_tool = [Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search")]

self_ask_with_search = initialize_agent(search_tool, llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True)

descriptionlist = []

for i in range(len(competitors_list)):
    company = competitors_list[i]
    searchstring = "Provide company information for " + company + ". Include company description and founders"
    result = self_ask_with_search.run(searchstring)
    descriptionlist.append(result)

print(descriptionlist)

In [ ]:
# Prompt Generation of hypothese for testing value proposition

hypotheses_template = PromptTemplate(
    input_variables = ['value_proposition', 'canvas'], 
    template='Generate a list of hypotheses for testing the following value proposition: {value_proposition}. You should rely on information from the value proposition canvas: {canvas}. Include at least 3 hypotheses. Each hypothesis should focus on the feasibility or viability of the business model. Here are some potential topics for hypotheses testing: willingness to pay, preferred distribution model, ultimate customer segmet, etc.')

hypotheses_chain = LLMChain(llm=llm, prompt=hypotheses_template, verbose=True, output_key='hypotheses')

hypotheses = hypotheses_chain.run(value_proposition=valueproposition, canvas=canvas)

print(hypotheses)

In [ ]:
# Write input and output to text file

with open('lean startup description.txt', 'w') as f:
    f.write('Input job to be done = ' + job_to_be_done + '\n')
    f.write('Input customer description = ' + customer_description + '\n')
    f.write('\n' + 'GPT output hardest part of job to be done = ' + hardest +  '\n')
    f.write('\n' + 'GPT output value proposition = ' + valueproposition +  '\n')
    f.write('\n' + 'GPT output value proposition canvas = ' + canvas +  '\n')
    f.write('\n' + 'GPT output most similar competitors = ' + competitors +  '\n')
    f.write('\n' + 'GPT output company description competitor 1 = ' + str(descriptionlist[0]) +  '\n')
    f.write('\n' + 'GPT output company description competitor 2 = ' + str(descriptionlist[1]) +  '\n')
    f.write('\n' + 'GPT output company description competitor 3 = ' + str(descriptionlist[2]) +  '\n')
    f.write('\n' + 'GPT output hypotheses = ' + hypotheses +  '\n')